# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## 1) Dataload 20newsgroups  data

In [1]:
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [5]:
X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [6]:
x_train=list(X_train)
x_test=list(X_test)

sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 300

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [7]:
# #마지막 out-of-vetor처리(정수 인덱스 처리된 단어 151049를 문장의 첫번째 단어로 보정) *이 방법이 아닌 unknown 토큰을 사용하는 방법도 있음
# xtext_test[-1][list(xtext_test[-1]).index(xtext_test[-1].max())]=xtext_test[-1][list(xtext_test[-1]).index(1)+1]

In [8]:
# 방법2) 케라스 내장함수 활용
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

## 2) 어텐션 레이어 정의

In [9]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs


class TransformerBlock(tf.keras.layers.Layer):
    
        
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm
    
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embedding_dim': self.embedding_dim,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'rate': self.rate
        })
        return config


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
    
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=max_len, output_dim=embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super(TokenAndPositionEmbedding, self).get_config()
        config.update({
            'max_len': self.max_len,
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim
        })
        return config

## 3) textcuboid 분류

In [10]:
textcuboid=np.load('./1-Channel textcuboid_20ng(bert).npy')

In [11]:
textcuboid_test=np.load('./1-Channel textcuboid_test_20ng(bert).npy')

In [12]:
import keras
from tensorflow.keras import layers

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [14]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers

embedding_dim = 256  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기

Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
Text_x = transformer_block(Text_x)
Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(256, activation="relu")(Text_x)

BERT_input = Input(shape=(155, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(BERT_input)
pooling = GlobalMaxPooling1D()(conv1)
BERT_x = Dense(1024, activation='relu')(pooling)


x = layers.concatenate([BERT_x, Text_x], axis=-1)
x = Dropout(0.5)(x)
output_layer = Dense(20, activation='softmax')(x)

model = Model(inputs=[BERT_input, Text_inputs], outputs=output_layer)

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 token_and_position_embedding (  (None, 300, 256)    38745600    ['input_1[0][0]']                
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 input_2 (InputLayer)           [(None, 155, 768)]   0           []                               
                                                                                                  
 transformer_block (Transformer  (None, 300, 256)    280864      ['token_and_position_embeddin

In [17]:
textcuboid_train=textcuboid[1000:]
text_train=xtext_train[1000:]
textcuboid__val=textcuboid[:1000]
text_val=xtext_train[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

text_test=xtext_test
y_test=one_hot_test_labels

In [18]:
history = model.fit([textcuboid_train,text_train], y_train,callbacks=callbacks, epochs=7,batch_size=256,validation_data=([textcuboid__val,text_val], y_val))

Epoch 1/7
55/55 [==============================] - 8s 120ms/step - loss: 3.8466 - accuracy: 0.2359 - val_loss: 1.5076 - val_accuracy: 0.5520 - lr: 0.0010
Epoch 2/7
55/55 [==============================] - 6s 106ms/step - loss: 1.1992 - accuracy: 0.6251 - val_loss: 0.7475 - val_accuracy: 0.7740 - lr: 0.0010
Epoch 3/7
55/55 [==============================] - 6s 107ms/step - loss: 0.4414 - accuracy: 0.8733 - val_loss: 0.4956 - val_accuracy: 0.8550 - lr: 0.0010
Epoch 4/7
55/55 [==============================] - 6s 103ms/step - loss: 0.1302 - accuracy: 0.9666 - val_loss: 0.4697 - val_accuracy: 0.8700 - lr: 0.0010
Epoch 5/7
55/55 [==============================] - 4s 78ms/step - loss: 0.0439 - accuracy: 0.9899 - val_loss: 0.5536 - val_accuracy: 0.8790 - lr: 0.0010
Epoch 6/7
55/55 [==============================] - 5s 82ms/step - loss: 0.0253 - accuracy: 0.9941 - val_loss: 0.5351 - val_accuracy: 0.8810 - lr: 0.0010
Epoch 7/7
55/55 [==============================] - 4s 81ms/step - loss: 0.0155

In [19]:
model.evaluate([textcuboid_test,text_test],y_test)

118/118 [==============================] - 1s 6ms/step - loss: 0.5761 - accuracy: 0.8821


[0.57611483335495, 0.8821030259132385]

In [20]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model.h5', custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)

118/118 [==============================] - 1s 7ms/step - loss: 0.4590 - accuracy: 0.8779


[0.4590297341346741, 0.8778544664382935]

In [21]:
np.average([88.5,87.1,87.9])

87.83333333333333